# Débruitage et inpainting d'image (modèle de Tikhonov)

In [ ]:
# import modules
import numpy as np
from math import *
import matplotlib.pyplot as plt
%matplotlib inline

## Quelques rappels 

### Descente de gradient

On rappelle que l'algorithme de descente de gradient consiste à construire la
suite récurrente

\begin{equation*}
  %
  \left\{
    %
    \begin{array}{cl}
      %
      x^0 &= \text{une initialisation donnée}\\
      x^{k+1} &= x^{k} - \alpha \nabla J(x^k) \quad (\text{pour tout }k \geq 0)
      %
    \end{array}
    %
  \right.
  %
\end{equation*}

afin d'approcher numériquement une solution du problème de minimisation de
l'énergie $J$ sur tout l'espace (pas de contraintes). Le paramètre $\alpha$ de ce
schéma est appelé _pas de descente_, et doit-être choisi "_assez-petit_" pour que l'algorithme converge.

### Gradient projeté

Dans le cas d'un problème avec contraintes, on cherche à minimiser $J$ sur un
ensemble de contraintes $\mathcal{C}$ (convexe), l'étape de mise à jour doit donc être
projeté dans l'ensemble des contraintes, conduisant à l'algorithme du
_gradient projeté_

\begin{equation*}
  %
  \left\{
    %
    \begin{array}{cl}
      %
      x^0 &= \text{une initialisation donnée}\\
      x^{k+1} &= \Pi_{\mathcal{C}}\left(x^{k} - \alpha \nabla J(x^k)\right)  \quad (\text{pour tout }k \geq 0)
      %
    \end{array}
    %
  \right.
  %
\end{equation*}

où $\Pi_\mathcal{C}$ désigne l'opérateur de projection sur le convexe $\mathcal{C}$.

## Implémentation d'un schéma itératif avec affichage de l'image à chaque itération

Lors de l'implémentation d'un schéma itératif, il est très utile d'afficher l'image $x^k$ ainsi que l'énergie $J(x^k)$ à chaque itération $k$ afin de contrôler que la minimisation se passe bien. 

Dans un Jupyter Notebook, le code ci-dessous peut-être utilisé (dans cet exemple, à chaque itération $k$, l'image $x^k$ s'obtient en ajoutant un bruit gaussien centré d'écart type $\sigma = 5 \cdot k$ à chaque pixel de l'image).


In [ ]:
# several additional imports
import pylab as pl
from IPython import display
import time

# example of iterative scheme with image display at each iteration
plt.figure(figsize=(7,7))
ref = np.double(plt.imread("artifice.tiff"))
for k in range(10):
    u = ref + np.random.normal(scale=5*k,size=ref.shape)
    plt.imshow(u,cmap='gray')
    display.clear_output(wait=True)
    display.display(pl.gcf())
    plt.title("itération %d : sig = %d" % (k,5*k))
    time.sleep(.5)
    

Cette méthode d'affichage ralentit malheureusement considérablement l'exécution du code, ce défaut est propre à l'utilisation de Jupyter notebook. En pratique on se contentera d'afficher l'image toutes les $N$ itérations (avec $N = 20$ ou $N=50$).

## Chargement de fonctions utiles (grad et div)

On définit les versions discrètes du gradient 
$$
 \nabla_d f(k) =
  \left\{
    %
    \begin{array}{cl}
      %
      f(k+1)-f(k) & \text{si } k+1<N \\
      0 & \text{sinon}.
      %
    \end{array}
    %
  \right.
$$
On a alors la relation usuelle $$\langle \nabla_d u ,P \rangle = \langle u , -\mathrm{div}_d(P) \rangle$$
qui est satisfaite, avec   
$$
\mathrm{div}_d(P)(k,\ell) = \left\{
    %
    \begin{array}{cll}
      %
      P_x(k,\ell) - P_x(k-1,\ell) &\text{si}& 1 \leq k \leq N_x-2 \\
      P_x(k,\ell) &\text{si}& k=0 \\
      -P_x(k-1,\ell) &\text{si}& k = N_x-1
      %
    \end{array}
    %
  \right.
  %
  ~+~
  %
  \left\{
    %
    \begin{array}{cll}
      %
      P_y(k,\ell)-P_y(k,\ell-1) & \text{si}& 1 \leq \ell \leq N_y-2 \\
      P_y(k,\ell) & \text{si}& \ell = 0\\
      -P_y(k,\ell-1) & \text{si}& \ell = N_y-1
      %
    \end{array}
    %
  \right.
$$


In [ ]:
# 2D discrete gradient 
def grad(u):
    ny,nx = u.shape
    Gx = np.zeros(u.shape)
    Gy = np.zeros(u.shape)
    Gx[:,0:nx-1] = u[:,1:nx] - u[:,0:nx-1]
    Gy[0:ny-1,:] = u[1:ny,:] - u[0:ny-1,:]
    return Gx,Gy

# divergence of a 2D discrete field
def div(px,py):
    ny,nx = px.shape
    # process the first component (px) of the input field
    div_x = np.zeros(px.shape)
    div_x[:,1:nx-1] = px[:,1:nx-1] - px[:,0:nx-2]
    div_x[:,0] = px[:,0]
    div_x[:,nx-1] = -px[:,nx-2]
    # process the second component (py) of the input field
    div_y = np.zeros(px.shape)
    div_y[1:ny-1,:] = py[1:ny-1,:] - py[0:ny-2,:]
    div_y[0,:] = py[0,:]
    div_y[ny-1,:] = -py[ny-2,:]
    # return output divergence
    return div_x + div_y

## Exercice 1. Débruitage d'images dégradées par un bruit gaussien

Supposons que l'on observe une image

$$u_0 = u + \varepsilon\,,$$

où $u$ représente l'image sans bruit (que l'on souhaite estimer) et
$\varepsilon$ une image dont les niveaux de gris suivent une loi
$\mathcal{N}(0,\sigma^2)$. Pour estimer $u$ à partir de $u_0$, nous proposons de
calculer


$$
u_\lambda=\text{argmin}_{x : \Omega \to \mathbb{R}}J(u) \quad \text{avec} \quad J(u)= \|u-u_0\|_2^2 + \lambda \|K u\|_2^2\,\qquad\qquad(\mathcal{P}) 
$$

où $\lambda >0$ et $K: \mathbb{R}^\Omega \to \mathbb{R}^\Omega$ est un opérateur linéaire.

1-a) **Optionnel (sauf pour le compte rendu de TP)** Montrer que le problème $(\mathcal{P})$ admet une unique solution.

1-b) **Optionnel (sauf pour le compte rendu de TP)** Résoudre $(\mathcal{P})$ dans le cas où $K=\mathrm{id}$, ce modèle présente-t-il un intérêt en terme de débruitage?

1-c) Dans le cas où $K = \nabla_d$, calculer $\nabla J$ puis approcher numériquement la solution de $(\mathcal{P})$ à l'aide d'un algorithme de descente de gradient. Vous prendrez soin d'évaluer l'énergie à minimiser à chaque itération de votre algorithme. Vous pourrez par exemple écrire une fonction
`gaussian_denoise(u0,lambda,niter,...)` qui renvoie l'image et l'energie calculée à chaque itération.

In [ ]:
### A COMPLETER 

1-d) Tester votre algorithme sur l'image `artifice.tiff` que vous dégraderez par ajout d'un bruit gaussien de moyenne nulle et d'écart-type $\sigma = 20$, contrôler la décroissance de l'énergie au fil des itérations.

In [ ]:
### A COMPLETER 

1-e) Comment varie la solution obtenue en fonction du paramètre $\lambda$?

1-f) **Optionnel (sauf pour le compte rendu de TP)** Quelle valeur du paramètre $\alpha$ suggérez-vous d'utiliser ? 

## Exercice 2. Extension à l'inpainting d'images

On peut adapter le problème précédent au cas plus général des problèmes
inverses, dans ce cas, on cherche à résoudre

\begin{equation}
  \text{argmin}_{u:\Omega\to\mathbb{R}} \|Au - u_0 \|_2^2 + \lambda \|\nabla_d u\|_2^2\,,\qquad\qquad(\mathcal{P}')
\end{equation}

où $A$ désigne l'opérateur linéaire que l'on souhaite "_inverser_". Dans le
cas de l'inpainting, l'opérateur $A$ est un opérateur de _masquage de
  pixels_, on considère ainsi que les niveaux de gris de l'image sont mesurés
seulement sur un sous domaine $\omega \subset \Omega$ de l'image et on pose

$$\forall(x,y) \in \Omega \,,\quad Au(x,y) = \left\{
  %
  \begin{array}{cl}
    %
    u(x,y) & \text{si } (x,y) \in \omega \\
    0 & \text{sinon,}
    %
  \end{array}
  %
\right.$$

il est facile de montrer que cet opérateur est auto-adjoint. Une implémentation de l'opérateur $A$ est proposée ci-dessous.

In [ ]:
# loading reference image
ref = np.double(plt.imread('artifice.tiff'))
ny,nx = ref.shape

# compute inpainting mask (around p percent of the pixels will be masked)
p = 70 
mask = np.random.rand(ny,nx) > p/100 # random mask

# define A operator (spatial masking operation)
def A(u):
    return u*mask

2-a) Écrire une variante de l'algorithme développé à l'exercice 1 pour résoudre le
problème $(\mathcal{P}')$.

In [ ]:
### A COMPLETER 

## Exercice 3. Débruitage d'images dégradées par un bruit de Poisson

Dans le cas où le bruit qui dégrade les données est un _bruit de Poisson_, le problème de débruitage
gaussien $(\mathcal{P})$ peut-être adapté au cas du bruit de Poisson de
la manière suivante

\begin{equation*}
  %
  \text{argmin}_{u : \Omega\to \mathbb{R}_+} \langle u - u_0 \log{u} , 1_\Omega \rangle + \lambda \|\nabla_d u\|_2^2\,. \qquad\qquad(\mathcal{P}^{\prime\prime})
  %
\end{equation*}

La simulation d'une image $u_0$ dégradée par un bruit de Poisson peut-être faite
le module `numpy.random.poisson`, comme indiqué ci-dessous.

In [ ]:
# loading reference image
ref = np.double(plt.imread('artifice.tiff'))

# compute an observation of 'ref' corrupted with Poisson noise
u0 = np.random.poisson(ref) 

# display reference and noisy image
fig = plt.figure(figsize=(15,15))

fig.add_subplot(1,2,1)
plt.imshow(ref,cmap='gray') 
plt.title('reference image')

fig.add_subplot(1,2,2)
plt.imshow(u0,cmap='gray') 
plt.title('noisy image')

3-a) Vérifier que plus les niveaux de gris de l'image de référence sont faibles (basse lumière), plus l'image est dégradée

3-b) **Optionnel :** Montrer que le gradient de $J(u) := \langle u - u_0 \log{u} , 1_\Omega \rangle$ est 
$$\nabla J(u)(x,y)=1_\Omega(x,y)-\left\{\begin{array}{cl}\frac{u_0(x,y)}{u(x,y)} & \text{si } (x,y)\in \omega_0 \\0 & \text{sinon.}\end{array}\right.$$
où 
$$\omega_0 = \{(x,y) \in \Omega,~u(x,y) > 0\}$$

3-c) Approcher numériquement la solution du problème $(\mathcal{P}^{\prime\prime})$ en
  utilisant l'algorithme du _gradient projeté_. Vous prendrez soin
  d'évaluer l'énergie à minimiser à chaque itération de l'algorithme.

In [ ]:
### A COMPLETER 

3-d) Utiliser l'algorithme de débruitage gaussien développé à l'exercice 1 pour débruiter une image dégradée par un bruit de Poisson, le résultat est-il satisfaisant?